In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder # import geocoder
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

# First Week
### Description of the problem: 
##### In this Project I will analize the venues and establishments, using the Foursquare API to get the datas and then I will apply Kmeans Clustering technique to try recognize a standard in the my home town called Pouso Alegre in Brazil. In my home town is common open the same kind of category of establishments in the same area, when it's occur one or both establishments doesn't support the compettion and ends up closing. In the final of this project I will be able to point where is the best place to open a new kind of establishments.


### Description of the data: 
##### Using the Foursquare API, I could get the latitude, longitude, category and the neighborhood for each establishments that has in my home town. This features it's enough to apply the Kmeans Cluster.


##### Getting the geograpical coordinate of my home towm

In [2]:
address = 'Pouso Alegre, BR'

geolocator = Nominatim(user_agent="PA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pouso Alegre City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pouso Alegre City are -22.2343858, -45.9327241.


#### Getting access to Foursquare API

In [3]:
CLIENT_ID = 'JJE4DQWK4PGQHG5IP0WKT3NKVRWLN2GAMLMLAM512RTYL4TD' # your Foursquare ID
CLIENT_SECRET = 'WOWFIYBACJEKQW0VEXJNQ0BHYR1JFESJWZ3NJXROLZTGWSQ0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JJE4DQWK4PGQHG5IP0WKT3NKVRWLN2GAMLMLAM512RTYL4TD
CLIENT_SECRET:WOWFIYBACJEKQW0VEXJNQ0BHYR1JFESJWZ3NJXROLZTGWSQ0


In [4]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=JJE4DQWK4PGQHG5IP0WKT3NKVRWLN2GAMLMLAM512RTYL4TD&client_secret=WOWFIYBACJEKQW0VEXJNQ0BHYR1JFESJWZ3NJXROLZTGWSQ0&v=20180604&ll=-22.2343858,-45.9327241&radius=500&limit=100'

#### Result of foursquare API

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d808ab866fc65002c6a4f39'},
 'response': {'headerLocation': 'Pouso Alegre',
  'headerFullLocation': 'Pouso Alegre',
  'headerLocationGranularity': 'city',
  'totalResults': 41,
  'suggestedBounds': {'ne': {'lat': -22.229885795499992,
    'lng': -45.92787169007461},
   'sw': {'lat': -22.238885804500004, 'lng': -45.93757650992539}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e73a604d16472c037433ca3',
       'name': 'Divina Maria Café & Cultura',
       'location': {'lat': -22.23347056429007,
        'lng': -45.93313954621554,
        'labeledLatLngs': [{'label': 'display',
          'lat': -22.23347056429007,
          'lng': -45.93313954621554}],
        'distance': 110,
        'cc': 'BR',
        'city': 'Pouso Alegre',
       

In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
results

{'meta': {'code': 200, 'requestId': '5d808ab866fc65002c6a4f39'},
 'response': {'headerLocation': 'Pouso Alegre',
  'headerFullLocation': 'Pouso Alegre',
  'headerLocationGranularity': 'city',
  'totalResults': 41,
  'suggestedBounds': {'ne': {'lat': -22.229885795499992,
    'lng': -45.92787169007461},
   'sw': {'lat': -22.238885804500004, 'lng': -45.93757650992539}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e73a604d16472c037433ca3',
       'name': 'Divina Maria Café & Cultura',
       'location': {'lat': -22.23347056429007,
        'lng': -45.93313954621554,
        'labeledLatLngs': [{'label': 'display',
          'lat': -22.23347056429007,
          'lng': -45.93313954621554}],
        'distance': 110,
        'cc': 'BR',
        'city': 'Pouso Alegre',
       

### Getting the venue characteristics

In [8]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Divina Maria Café & Cultura,Café,-22.233471,-45.933140
1,República Bar,Nightclub,-22.233757,-45.933293
2,Vila do Pão De Queijo,Brazilian Restaurant,-22.232912,-45.933603
3,Padre Copinho,Gastropub,-22.233357,-45.933695
4,Café Portal,Café,-22.232875,-45.934647


##### And how many venues were returned by Foursquare?

In [9]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

41 venues were returned by Foursquare.


# Second Week

## Introduction
#### I had discussed this topics with some merchants in my home town, They will be interested in this project, because with this results we can choose the best place to open a new venue, it will be possible to point where is missing determined particular market type and optimize the sales.


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
PA_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng'])

Divina Maria Café & Cultura
República Bar
Vila do Pão De Queijo
Padre Copinho
Café Portal
Emporio Minas Sul
Maracujá com Manga
Uyrapuru Restaurante e Cantina
Hering Store Pouso Alegre
Oficina de Pizza
Arlen Hotel
Mr. Mix
Academia Ana Maria
Kanguroo's
4YOU Brasil
Restaurante O Caipira
Namastê
Teatro Municipal
Cacau Show
Feira Livre
Avenida Grill
Cacau Show
Restaurante Vila Rica
Tim Pouso Alegre
Supermercados Alvorada
Casa do Açaí
Churrascaria e Restaurante Beverly
Grande Hotel Zé Maria
Hotel Dias
Fenix Hotel
Magazine Luiza
VIP lanches
Bola De Ouro
Ribs Station
Só Na Brasa Espetinhos
Restaurante Tempero Caseiro
Bob's
Emporio Sta Luzia
BUCK Alimentos E Serviços Terminal Rodoviário
Terminal Rodoviário de Pouso Alegre
Dique 2


#### Getting the completed dataframe with the coordinate of Neighborhood and coordinate of Venues.

In [12]:
PA_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Divina Maria Café & Cultura,-22.233471,-45.93314,Café Portal,-22.232875,-45.934647,Café
1,Divina Maria Café & Cultura,-22.233471,-45.93314,Divina Maria Café & Cultura,-22.233471,-45.933140,Café
2,Divina Maria Café & Cultura,-22.233471,-45.93314,Maracujá com Manga,-22.233036,-45.935116,Wine Bar
3,Divina Maria Café & Cultura,-22.233471,-45.93314,Uyrapuru Restaurante e Cantina,-22.232098,-45.934943,Mineiro Restaurant
4,Divina Maria Café & Cultura,-22.233471,-45.93314,República Bar,-22.233757,-45.933293,Nightclub


In [13]:
# one hot encoding
PA_onehot = pd.get_dummies(PA_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
PA_onehot['Neighborhood'] = PA_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [PA_onehot.columns[-1]] + list(PA_onehot.columns[:-1])
PA_onehot = PA_onehot[fixed_columns]

PA_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Café,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Diner,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food Truck,Gastropub,Gourmet Shop,Grocery Store,Gym / Fitness Center,Gymnastics Gym,Historic Site,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Mineiro Restaurant,Movie Theater,Nightclub,Pedestrian Plaza,Pizza Place,Plaza,Restaurant,Sandwich Place,Stadium,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,Divina Maria Café & Cultura,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Divina Maria Café & Cultura,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Divina Maria Café & Cultura,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Divina Maria Café & Cultura,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Divina Maria Café & Cultura,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category, this point it's very important to avoid the initial problem.

In [14]:
PA_grouped = PA_onehot.groupby('Neighborhood').mean().reset_index()
PA_grouped.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Café,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Diner,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food Truck,Gastropub,Gourmet Shop,Grocery Store,Gym / Fitness Center,Gymnastics Gym,Historic Site,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Mineiro Restaurant,Movie Theater,Nightclub,Pedestrian Plaza,Pizza Place,Plaza,Restaurant,Sandwich Place,Stadium,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,4YOU Brasil,0.000000,0.000000,0.000000,0.040000,0.120000,0.0,0.000000,0.000000,0.040000,0.000000,0.040000,0.04,0.000000,0.000,0.000000,0.000000,0.000000,0.04,0.000000,0.000000,0.040000,0.04,0.000000,0.000000,0.080000,0.000000,0.040000,0.000000,0.04,0.000000,0.120000,0.040000,0.000000,0.040000,0.040000,0.000000,0.000000,0.040000,0.000000,0.080000,0.000000,0.040000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000
1,Academia Ana Maria,0.026316,0.026316,0.026316,0.026316,0.078947,0.0,0.026316,0.026316,0.052632,0.052632,0.026316,0.00,0.000000,0.000,0.026316,0.026316,0.052632,0.00,0.000000,0.026316,0.052632,0.00,0.000000,0.026316,0.052632,0.026316,0.026316,0.026316,0.00,0.000000,0.052632,0.000000,0.026316,0.000000,0.000000,0.026316,0.000000,0.026316,0.026316,0.026316,0.000000,0.026316,0.00,0.000000,0.026316,0.026316,0.000000,0.026316
2,Arlen Hotel,0.025000,0.000000,0.025000,0.025000,0.075000,0.0,0.025000,0.025000,0.050000,0.025000,0.025000,0.00,0.025000,0.025,0.000000,0.025000,0.025000,0.00,0.000000,0.025000,0.050000,0.00,0.000000,0.025000,0.050000,0.025000,0.025000,0.025000,0.00,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.025000,0.025000,0.025000,0.000000,0.050000,0.00,0.025000,0.025000,0.025000,0.000000,0.025000
3,Avenida Grill,0.021277,0.021277,0.021277,0.021277,0.085106,0.0,0.000000,0.000000,0.042553,0.042553,0.042553,0.00,0.000000,0.000,0.021277,0.021277,0.042553,0.00,0.021277,0.021277,0.063830,0.00,0.021277,0.021277,0.042553,0.021277,0.021277,0.021277,0.00,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.000000,0.021277,0.021277,0.042553,0.00,0.000000,0.021277,0.021277,0.021277,0.021277
4,BUCK Alimentos E Serviços Terminal Rodoviário,0.027778,0.000000,0.027778,0.027778,0.083333,0.0,0.027778,0.027778,0.055556,0.027778,0.000000,0.00,0.027778,0.000,0.000000,0.027778,0.055556,0.00,0.000000,0.027778,0.055556,0.00,0.000000,0.027778,0.055556,0.000000,0.027778,0.027778,0.00,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.027778,0.027778,0.027778,0.000000,0.055556,0.00,0.027778,0.027778,0.027778,0.000000,0.027778


In [15]:
PA_grouped.shape

(40, 49)

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = PA_grouped['Neighborhood']

for ind in np.arange(PA_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(PA_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4YOU Brasil,Brazilian Restaurant,Hotel,Gastropub,Pizza Place,Café,Japanese Restaurant,Fast Food Restaurant,Fish & Chips Shop,Coffee Shop,Grocery Store
1,Academia Ana Maria,Brazilian Restaurant,Hotel,Gastropub,Fast Food Restaurant,Café,Chocolate Shop,Dessert Shop,Wine Bar,Department Store,Gourmet Shop
2,Arlen Hotel,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Wine Bar,Convenience Store,Farmers Market,Dessert Shop
3,Avenida Grill,Brazilian Restaurant,Fast Food Restaurant,Gastropub,Restaurant,Café,Chocolate Shop,Clothing Store,Dessert Shop,Wine Bar,Event Space
4,BUCK Alimentos E Serviços Terminal Rodoviário,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Dessert Shop,Department Store,Grocery Store,Farmers Market


### I choose the K-means Cluster technique, because this is a unsupervised, that is unlabeled data.

In [18]:
# set number of clusters
kclusters = 5

PA_grouped_clustering = PA_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(PA_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 1, 0, 3, 1, 1, 0, 0, 4])

In [19]:
PA_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Divina Maria Café & Cultura,-22.233471,-45.93314,Café Portal,-22.232875,-45.934647,Café
1,Divina Maria Café & Cultura,-22.233471,-45.93314,Divina Maria Café & Cultura,-22.233471,-45.933140,Café
2,Divina Maria Café & Cultura,-22.233471,-45.93314,Maracujá com Manga,-22.233036,-45.935116,Wine Bar
3,Divina Maria Café & Cultura,-22.233471,-45.93314,Uyrapuru Restaurante e Cantina,-22.232098,-45.934943,Mineiro Restaurant
4,Divina Maria Café & Cultura,-22.233471,-45.93314,República Bar,-22.233757,-45.933293,Nightclub


#### Getting the unique PA_Neighborhood

In [20]:
PA_Neighborhood = PA_venues.groupby(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude',]).size().reset_index(name='Freq')
PA_Neighborhood.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Freq
0,4YOU Brasil,-22.231611,-45.929270,25
1,Academia Ana Maria,-22.234130,-45.933681,38
2,Arlen Hotel,-22.235197,-45.933162,40
3,Avenida Grill,-22.232199,-45.934812,47
4,BUCK Alimentos E Serviços Terminal Rodoviário,-22.236475,-45.935552,36


In [21]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4YOU Brasil,Brazilian Restaurant,Hotel,Gastropub,Pizza Place,Café,Japanese Restaurant,Fast Food Restaurant,Fish & Chips Shop,Coffee Shop,Grocery Store
1,Academia Ana Maria,Brazilian Restaurant,Hotel,Gastropub,Fast Food Restaurant,Café,Chocolate Shop,Dessert Shop,Wine Bar,Department Store,Gourmet Shop
2,Arlen Hotel,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Wine Bar,Convenience Store,Farmers Market,Dessert Shop
3,Avenida Grill,Brazilian Restaurant,Fast Food Restaurant,Gastropub,Restaurant,Café,Chocolate Shop,Clothing Store,Dessert Shop,Wine Bar,Event Space
4,BUCK Alimentos E Serviços Terminal Rodoviário,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Dessert Shop,Department Store,Grocery Store,Farmers Market


### This is where we got the data that will be possible to solve the problem.

 ##### Creating a Merged Dataframe containing the Neighborhood and the the most common venues. 

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

PA_merged = PA_Neighborhood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
PA_merged = PA_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

PA_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Freq,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4YOU Brasil,-22.231611,-45.929270,25,2,Brazilian Restaurant,Hotel,Gastropub,Pizza Place,Café,Japanese Restaurant,Fast Food Restaurant,Fish & Chips Shop,Coffee Shop,Grocery Store
1,Academia Ana Maria,-22.234130,-45.933681,38,4,Brazilian Restaurant,Hotel,Gastropub,Fast Food Restaurant,Café,Chocolate Shop,Dessert Shop,Wine Bar,Department Store,Gourmet Shop
2,Arlen Hotel,-22.235197,-45.933162,40,1,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Wine Bar,Convenience Store,Farmers Market,Dessert Shop
3,Avenida Grill,-22.232199,-45.934812,47,0,Brazilian Restaurant,Fast Food Restaurant,Gastropub,Restaurant,Café,Chocolate Shop,Clothing Store,Dessert Shop,Wine Bar,Event Space
4,BUCK Alimentos E Serviços Terminal Rodoviário,-22.236475,-45.935552,36,3,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Dessert Shop,Department Store,Grocery Store,Farmers Market


In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(PA_merged['Neighborhood Latitude'], PA_merged['Neighborhood Longitude'], PA_merged['Neighborhood'], PA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

### Let's examine each cluster and determine the discriminating venue categories that distinguish each cluster, this will avoid to point a place that doesn't have a perfil for your estabilishment.

## Results section 
#### I got the most common venue in each area, so it's important avoid the most common venues in this areas for each cluster. Otherwise we can fall in the same initial showed, that is when your market type doesn't fit in the area.

## recommendations
#### So in the first cluster is recommended to choose a Dessert shop, Wine Bar or an Event Space in the Avenida Grill Area and so on.

#### Cluster 1

In [24]:
PA_merged.loc[PA_merged['Cluster Labels'] == 0, PA_merged.columns[[0] + list(range(5, PA_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Avenida Grill,Brazilian Restaurant,Fast Food Restaurant,Gastropub,Restaurant,Café,Chocolate Shop,Clothing Store,Dessert Shop,Wine Bar,Event Space
7,Cacau Show,Brazilian Restaurant,Fast Food Restaurant,Dessert Shop,Clothing Store,Chocolate Shop,Restaurant,Café,Gastropub,Wine Bar,Cosmetics Shop
8,Cacau Show,Brazilian Restaurant,Fast Food Restaurant,Dessert Shop,Clothing Store,Chocolate Shop,Restaurant,Café,Gastropub,Wine Bar,Cosmetics Shop
9,Café Portal,Brazilian Restaurant,Fast Food Restaurant,Chocolate Shop,Café,Hotel,Dessert Shop,Clothing Store,Gastropub,Restaurant,Bus Station
11,Churrascaria e Restaurante Beverly,Brazilian Restaurant,Fast Food Restaurant,Chocolate Shop,Pizza Place,Hotel,Dessert Shop,Clothing Store,Café,Gastropub,Restaurant
17,Fenix Hotel,Brazilian Restaurant,Fast Food Restaurant,Dessert Shop,Hotel,Gastropub,Café,Chocolate Shop,Clothing Store,Wine Bar,Gourmet Shop
19,Hering Store Pouso Alegre,Brazilian Restaurant,Fast Food Restaurant,Gastropub,Café,Restaurant,Chocolate Shop,Clothing Store,Dessert Shop,Wine Bar,Farmers Market
20,Hotel Dias,Brazilian Restaurant,Fast Food Restaurant,Hotel,Café,Restaurant,Chocolate Shop,Clothing Store,Gastropub,Dessert Shop,Wine Bar
21,Kanguroo's,Brazilian Restaurant,Fast Food Restaurant,Clothing Store,Gastropub,Hotel,Café,Chocolate Shop,Dessert Shop,Department Store,Flea Market
22,Magazine Luiza,Brazilian Restaurant,Fast Food Restaurant,Gastropub,Café,Restaurant,Chocolate Shop,Clothing Store,Dessert Shop,Wine Bar,Farmers Market


#### Cluster 2

In [25]:
PA_merged.loc[PA_merged['Cluster Labels'] == 1, PA_merged.columns[[0] + list(range(5, PA_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Arlen Hotel,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Wine Bar,Convenience Store,Farmers Market,Dessert Shop
5,Bob's,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Wine Bar,Convenience Store,Farmers Market,Dessert Shop
6,Bola De Ouro,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Dessert Shop,Wine Bar,Convenience Store,Farmers Market
15,Emporio Sta Luzia,Hotel,Brazilian Restaurant,Clothing Store,Gastropub,Café,Restaurant,Chocolate Shop,Pizza Place,Fast Food Restaurant,Department Store
16,Feira Livre,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Dessert Shop,Wine Bar,Convenience Store,Farmers Market
32,Ribs Station,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Wine Bar,Convenience Store,Farmers Market,Dessert Shop
33,Supermercados Alvorada,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Chocolate Shop,Wine Bar,Convenience Store,Farmers Market
34,Só Na Brasa Espetinhos,Hotel,Brazilian Restaurant,Gastropub,Restaurant,Fast Food Restaurant,Café,Wine Bar,Department Store,Gourmet Shop,Farmers Market


#### Cluster 3

In [26]:
PA_merged.loc[PA_merged['Cluster Labels'] == 2, PA_merged.columns[[0] + list(range(5, PA_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4YOU Brasil,Brazilian Restaurant,Hotel,Gastropub,Pizza Place,Café,Japanese Restaurant,Fast Food Restaurant,Fish & Chips Shop,Coffee Shop,Grocery Store


#### Cluster 4

In [27]:
PA_merged.loc[PA_merged['Cluster Labels'] == 3, PA_merged.columns[[0] + list(range(5, PA_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,BUCK Alimentos E Serviços Terminal Rodoviário,Hotel,Brazilian Restaurant,Gastropub,Fast Food Restaurant,Restaurant,Café,Dessert Shop,Department Store,Grocery Store,Farmers Market
12,Dique 2,Hotel,Brazilian Restaurant,Café,Fast Food Restaurant,Gastropub,Wine Bar,Bakery,Burger Joint,Bus Station,Comfort Food Restaurant
18,Grande Hotel Zé Maria,Hotel,Brazilian Restaurant,Gastropub,Restaurant,Café,Fast Food Restaurant,Dessert Shop,Grocery Store,Gourmet Shop,Farmers Market
26,Oficina de Pizza,Brazilian Restaurant,Dessert Shop,Gastropub,Fast Food Restaurant,Café,Hotel,Wine Bar,Grocery Store,Video Game Store,Farmers Market
36,Terminal Rodoviário de Pouso Alegre,Brazilian Restaurant,Hotel,Dessert Shop,Gastropub,Fast Food Restaurant,Restaurant,Café,Wine Bar,Grocery Store,Farmers Market
39,VIP lanches,Hotel,Brazilian Restaurant,Café,Fast Food Restaurant,Gastropub,Wine Bar,Bakery,Burger Joint,Bus Station,Chocolate Shop


#### Cluster 5

In [28]:
PA_merged.loc[PA_merged['Cluster Labels'] == 4, PA_merged.columns[[0] + list(range(5, PA_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Academia Ana Maria,Brazilian Restaurant,Hotel,Gastropub,Fast Food Restaurant,Café,Chocolate Shop,Dessert Shop,Wine Bar,Department Store,Gourmet Shop
10,Casa do Açaí,Brazilian Restaurant,Hotel,Gastropub,Fast Food Restaurant,Café,Chocolate Shop,Dessert Shop,Wine Bar,Department Store,Gourmet Shop
13,Divina Maria Café & Cultura,Brazilian Restaurant,Clothing Store,Gastropub,Hotel,Fast Food Restaurant,Dessert Shop,Café,Chocolate Shop,Cosmetics Shop,Farmers Market
14,Emporio Minas Sul,Fast Food Restaurant,Brazilian Restaurant,Clothing Store,Gastropub,Hotel,Dessert Shop,Café,Chocolate Shop,Cosmetics Shop,Farmers Market
25,Namastê,Brazilian Restaurant,Chocolate Shop,Hotel,Gastropub,Fast Food Restaurant,Café,Dessert Shop,Department Store,Gourmet Shop,Farmers Market
27,Padre Copinho,Brazilian Restaurant,Clothing Store,Gastropub,Hotel,Fast Food Restaurant,Dessert Shop,Café,Chocolate Shop,Cosmetics Shop,Farmers Market
28,República Bar,Brazilian Restaurant,Gastropub,Hotel,Fast Food Restaurant,Dessert Shop,Café,Chocolate Shop,Wine Bar,Cosmetics Shop,Farmers Market
29,Restaurante O Caipira,Brazilian Restaurant,Gastropub,Hotel,Fast Food Restaurant,Dessert Shop,Café,Chocolate Shop,Wine Bar,Cosmetics Shop,Farmers Market
31,Restaurante Vila Rica,Brazilian Restaurant,Gastropub,Hotel,Fast Food Restaurant,Café,Chocolate Shop,Dessert Shop,Wine Bar,Department Store,Video Game Store


# Conclusion
#### In this Project I could obtain the most common venue, market type in each commercial area in my home town, it's important know that to avoid to open a new market type that is already saturated in this region. In summary, I could identify what market type avoid and look for each commercial region in Brazil, Pouso Alegre - MG.